In [2]:
import grab
import logging
from grab.spider import Task, Spider

In [19]:
import pandas as pd

In [3]:
import json

In [8]:
class ReviewSpider(Spider):
    # С этого списка начинается краулинг.
    # Дальше мы можем добавлять задания на основе контента каждого элемента списка (но может быть и один, конечно)
    initial_urls = ['https://play.google.com/store/apps/category/FAMILY?age=AGE_RANGE1&hl=ru']
    # список приложений на Google Play до 5 лет 
    # для каждого элемента списка initial_urls автоматически создается новый объект класса Task с параметром initial

    def prepare(self):
        # Здесь мы можем подготовить все, что нам надо: соединиться с базой данных, открыть нужные файлы для записи и так далее
        # Мы будем писать результаты в виде текста на диск
        self.results = []
        self.counter = 1
        self.template = 'news_{}.txt'
    
    # Вытащим ссылки на приложения из страницы
    def task_initial(self, grab, task):
        # Здесь происходит обработка изначальных урлов и отсюда генерируются новые задачи
        # task передается этой функции которая вызывается внутри grab
        # у любого обработчика должны быть аргументы задачи - grab - все скаченное и task - св-во события (номер документа или в 
        # в нашем случае url) и когда мы создаем эту инструкцию, для каждого элемента создаем новое событие 
        
        logging.debug ('Обработка изначальных URLов')
        for elem in grab.doc.select('//div[@class="details"]//a[contains(@class, "title")]'):
            # Для каждой статьи создадим задачу другого типа, которая будет обработана функцией task_<название_типа>
            yield Task('post', url=grab.make_url_absolute(elem.attr('href'))) 
            
            # новый эксземпляр Task типа таск
            # attr - позволяет за счет атрибута вытащить значение атрибута как индекс
            # make_url_absolute - не относительный, а полный url 
            # 'post' - название которые мы дали всей задачи и оно напрямую связано с названием task_post(self, grab, task)
    
    #Вытащим тексты отзывов для каждой ссылки  
    # отдельная функция - отдельная страница 
    def task_post(self, grab, task):
        logging.debug ('Скачиваем review по адресу {}'.format(task.url)) ## это для ошибок ???
        
        # один блок  @class = "single-review"

        review_block = grab.doc.select('//div[@class = "single-review"]') 
        review_text = review_block.select('//div[@class="review-body with-review-wrapper"]') 
        review_date = review_block.select('//div[@class="review-info"]//span[@class="review-date"]')
        
        text_list = []
        date_list = []
        
        for review in review_text:
            text = '{}'.format(review.text() if len(review_text) else 'no text')
            text_list.append(text)
        
        for date in review_date:
            date_ = '{}'.format(date.text() if len(review_date) else 'no date')
            date_list.append(date_)
                
        # список элементов, у которых есть свои методы и атрибуты 
        # условия равно ReviewText-  отзыв 
        # review_text - это по формату список, и хотя мы получаем всего один элемент в список, 
        # дальше мы обращаемся к нему, как к списку review_text[0]
        
        path = self.template.format(self.counter)
        grab.doc.save(path)

        
        with(open(path, 'w', encoding='utf-8')) as out_file:
            #out_file.write(''.join(text_list))
            json.dump(({'text': text_list, 'url': task.url}), out_file)
            #сохраняя файл в json  в виде словаря с двумя ключами 'text' и 'url'
            #, 'date': date_list
            
        self.counter += 1

In [9]:
bot = ReviewSpider(thread_number=2) # экзмепляр класса на практике 

# thread_number - кол-во потоков 
# многопоточное программирование - внутри программы может работать несколько потоков 
# или скачивание сразу 2-х отзывов 
# чем больше потоков, тем больше нагрузка на сеть 

In [10]:
bot.run() # запускается grab - есть очеред скачивание, дальше кладет в очередь новые задачи, сначали типа 0
# сначала кладет по одной задаче в очередь из initial_urls 
# дальше || если очередь не пустая он берет 1 элемент и запускает обработчик типа 

## Объединяем все файлы 

In [12]:
import glob2

In [13]:
filenames = glob2.glob('*.txt')  # list of all .txt files in the directory

www = {}  #key - urls, values -texts

for file in filenames:
    if file.startswith('news'):
        with open(file) as inp_file:
            data = json.load(inp_file)
            www[data['url']] = data['text'] #(data['date'], ) #создаем их двух ключей словаря, один ключ и значение

with open('outfile_json.txt', 'w') as f: 
    json.dump(www, f, sort_keys=True, indent=2) #indent - отступы 

In [4]:
with open('news_1.txt', 'r') as ff: 
    data_1 = json.load(ff)

In [14]:
with open('outfile_json.txt', 'r') as f: 
    data_new = json.load(f)

## Создаем dataframe 

In [87]:
df_text = pd.DataFrame()

In [88]:
text_review = []
for x in data_new.values():
    for k in x:
        k=k.replace('Читать дальше',"")
        if len(k) > 35:
            text_review.append(k)
        else:
            break

In [96]:
text_review_all = [x for x in text_review]

In [97]:
df_text['review'] = text_review_all

In [98]:
df_text.tail()

,review
13050,Ребенку очень нравится. Играет с судовольствием.
13051,"Дочке 2 года, очень нравится. Красочно, забавн..."
13052,Харошая игра маему ребёнку понравилось и мне т...
13053,"Игра интересная, ребенку очень нравится. Но по..."
13054,"Очень хорошее, качественное озвученное приложе..."


In [92]:
len(df_text.review[1])

42

In [93]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13055 entries, 0 to 13054
Data columns (total 1 columns):
review    13055 non-null object
dtypes: object(1)
memory usage: 102.1+ KB


In [ ]:
## Сохраняем DF в pickle 

In [95]:
import pickle

In [99]:
with open('text_review_all.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(text_review_all, f, pickle.HIGHEST_PROTOCOL)